In [26]:
from nltk.stem.snowball import SnowballStemmer as porter2
import numpy as np
import re
import math
from string import punctuation
import xml.etree.ElementTree as ET
import os

# loads text file data line by line into a list
def txt_loader(file):
    wrapper = open(file,"r",encoding="utf-8-sig")
    data = wrapper.readlines()
    data = [line.strip() for line in data if line.strip()]
    return data

# preprocessor module: performs tokenisation, case-folding, stop-word removal and stemming.
# set bool_flag=True so as not to remove stop-words 'and, 'or' and 'not' from a boolean query
def preprocessor(raw,bool_flag=False):
    data = re.split(r"[^a-zA-Z0-9']+",raw)
    data = [re.sub(r"^'|'$",'',s) for s in data]
    data = [tok.lower() for tok in data if len(tok)>0]
    stop_words = txt_loader("englishST.txt") 
    if bool_flag:
        stop_words = set(stop_words) - set(['and','not','or'])
    data = [tok for tok in data if tok not in stop_words]
    stemmer = porter2("english")
    data = [stemmer.stem(tok) for tok in data]
    return data

# parses the XML tree returning a list of tuples representing our documents,
# where tuple[0] is the document headline and text combined, and t[1] is the document's ID
def xml_parse(root):
    docIDs = []
    headlines = []
    text = []
    for child in root.iter():
        if child.tag == 'DOCNO':
            docID = int(child.text)
            docIDs.append(docID)
        if child.tag == 'HEADLINE':
            headlines.append(child.text)
        if child.tag == 'TEXT':
            text.append(child.text)
    docs = [(h + t,ID) for h, t, ID in zip(headlines, text, docIDs)]
    return docs

# creates a positional inverted index from a list of parsed documents
def create_index(docs):    
    idx = {}
    for c1, d in enumerate(docs):
        tokens = preprocessor(d[0])
        for c2, tok in enumerate(tokens):
            if tok in idx:
                prior = idx.get(tok)
                idx[tok] = prior + [(d[1],c2)]
            if tok not in idx:
                idx[tok] = [(d[1],c2)]
    return idx

# returns all docIDs containing the input term, set negate_flag=True if returning all docIDs NOT containing term
def bool_search(term,negate_flag=False):    
    if term not in idx:
        return []
    ids = set([a[0] for a in idx[term]])
    if negate_flag:
        return sorted(set(docIDs)-ids)
    else:
        return sorted(ids)

# returns docIDs containing the phrase given by 'term1 term2'
def phrase_search(term1, term2):    
    match_docs =[]
    overlapIDs = intersection(bool_search(term1),bool_search(term2))
    postings_1 = [i for i in idx[term1] if i[0] in overlapIDs]
    postings_2 = [i for i in idx[term2] if i[0] in overlapIDs]
    
    for i in overlapIDs:
        tpositions_1 = [a[1] for a in postings_1 if a[0]==i]
        tpositions_2 = [a[1]-1 for a in postings_2 if a[0]==i]
        if any(item in tpositions_1 for item in tpositions_2):
            match_docs.append(i)
            
    return match_docs

# returns docIDs for which term1 and term2 are at most d positions apart, after stop words have been removed
def proximity_search(d,term1, term2):
    d = int(d)
    match_docs =[]
    overlapIDs = intersection(bool_search(term1),bool_search(term2))
    postings_1 = [i for i in idx[term1] if i[0] in overlapIDs]
    postings_2 = [i for i in idx[term2] if i[0] in overlapIDs]
    
    for i in overlapIDs:
        tpositions_1 = [a[1] for a in postings_1 if a[0]==i]
        tpositions_2 = [a[1] for a in postings_2 if a[0]==i]
        prox_check = [True if abs(x-y)<=d else False for x in tpositions_1 for y in tpositions_2]
        if any(prox_check):
            match_docs.append(i)
            
    return match_docs

# Returns the common docIDs between two searches
def intersection(list1,list2):
    return sorted(list(set(list1).intersection(list2)))

# Returns all docIDs retrieved by two searches
def union(list1,list2):
    return sorted(list(set(list1).union(list2)))

# returns the list of docIDs for which the query occurs in
def search(query):
    # simple term search
    if len(query)==1:
        return bool_search(query[0])
    # simple term search with negation
    if len(query)==2 and query[0]=='not':
        return bool_search(query[1],True)
    # phrase search
    if len(query)==2:
        return phrase_search(query[0],query[1])
    # first token is a number + absence of boolean operator but length of query>2 implies proximity search 
    if query[0].isnumeric() and 'or' not in query[1:] and 'and' not in query[1:]:
        return proximity_search(query[0],query[1], query[2])
    # otherwise search query contains boolean operators, which are identified by flags
    and_flag = False
    or_flag = False
    if 'and' in query:
        and_flag = True
        split = query.index('and')
    if 'or' in query:
        or_flag = True
        split = query.index('or')
    # query is split where the boolean operator occurs, treated as separate queries then merged    
    first = query[:split]
    second = query[split+1:]  
    res1=[]
    res2=[]
    if len(first)==1:
        res1 = bool_search(first[0])
    if len(second)==1:
        res2 = bool_search(second[0])
    if len(first)==2 and 'not' in first:
        res1 = bool_search(first[1],True)
    elif len(first)==2:
        res1 = phrase_search(first[0],first[1])
    if len(second)==2 and 'not' in second:
        res2 = bool_search(second[1],True) 
    elif len(second)==2 and 'not' not in second:
        res2 = phrase_search(second[0],second[1])
    # merged by union or intersection depending on which operator was called
    if and_flag:
        return intersection(res1,res2)
    if or_flag:
        return union(res1,res2)

# returns the count of appearances of a term in a document
def tf(term,docID):
    return len([a[0] for a in idx[term] if a[0]==docID])

# returns the total number of documents containing the term
def df(term):
    return len(set([a[0] for a in idx[term]]))

# returns the inverse domain frequency of a term
def idf(term):
    return np.log10(len(docIDs)/df(term))

# returns the score based on TDIDF weight for a query phrase
def score(query,docID):
    return sum([(1+np.log10(tf(term,docID)))*idf(term) for term in query if tf(term,docID)>0])

# sorts docID and score pairs by score to give highest scoring docID first for a given query
def rank(query):
    return sorted([(docID,score(query,docID)) for docID in docIDs if score(query,docID)>0],key = lambda x: x[1], reverse=True)

# writes the positional inverted index to "index.txt" in the specified format
def index_out():
    f = open("index.txt",'w+')
    for term in idx:
        doc_freq = df(term)
        f1 = sorted([(b,[a[1] for a in idx[term] if a[0]==b]) for b in set([a[0] for a in idx[term]])], key = lambda x : x[0])

        f.write("{term}:{df}\n".format(term=term,df=doc_freq))
        for i, posting_list in enumerate(f1):
            f.write("\t{docID}: ".format(docID=f1[i][0]))
            for x, position in enumerate(f1[i][1]):
                if x == len(f1[i][1])-1:
                    f.write("{pos}\n".format(pos = position))
                    break
                f.write("{pos},".format(pos = position))
    f.close()
    return

# writes all positive resulting docIDs per query to "results.boolean.txt"
def b_results(file):
    f = open("results.boolean.txt","w+")
    b_queries = txt_loader(file)
    b_processed = [preprocessor(query,True) for query in b_queries]
    b_processed = [q[1:] for q in b_processed]
    for i,x in enumerate(b_processed,1):
        for j in search(x):
            f.write("{query},{docID}\n".format(query=i,docID=j))
    f.close()
    return

# writes at most 150 docIDs and their score for a given query to "results.ranked.txt" with highest scoring first
def r_results(file):
    f = open("results.ranked.txt","w+")
    r_queries = txt_loader(file)
    r_processed = [preprocessor(q) for q in r_queries]
    # remove query number from query
    r_processed = [q[1:] for q in r_processed]
    out=[]
    for i,x in enumerate(r_processed,1):
        for c,j in enumerate(rank(x)):
            if c == 150:
                break
            ad = [i] + list(j)
            out.append(ad)
    out = [[a[0],a[1],round(a[2],4)] for a in out]
    for en in out:
        f.write("{query},{docID},{score}\n".format(query=en[0],docID=en[1],score=en[2]))
    f.close()
    return

# create global variables

root = ET.parse(os.getcwd() + '/trec.5000.xml').getroot()
docs = xml_parse(root)
idx = create_index(docs)

docIDs = [d[1] for d in docs]
terms = list(idx.keys())

index_out()
b_results("queries.boolean.txt")
r_results("queries.ranked.txt")

Estimated runtime:172.12s
